In [2]:
# Import libraries
import requests
import pandas as pd
import re

In [3]:
# Create an URL object
url = 'https://en.wikipedia.org/wiki/English_football_clubs_in_international_competitions'

In [4]:
# Create object page
page = requests.get(url)

In [5]:
df = pd.read_html(url)

In [6]:
#Check the correct table is being picked up
df[3]

,Season,Club,Progress,Score,Opponents,Venue(s)
0,1955–56,None entered,None entered,None entered,None entered,None entered
1,1956–57,Manchester United,Semi-finals,3–5,Real Madrid,1–3 at Santiago Bernabéu2–2 at Old Trafford
2,1957–58,Manchester United,Semi-finals,2–5,Milan,2–1 at Old Trafford0–4 at San Siro
3,1958–59,Manchester United,First round,NaN,Young Boys,Walkover – United withdrawn by the Football Le...
4,1958–59,Wolverhampton Wanderers,First round,3–4,Schalke 04,2–2 at Molineux1–2 at Glückauf-Kampfbahn
...,...,...,...,...,...,...
139,2020–21,Chelsea,Winners,1–0,Manchester City,Estádio do Dragão
140,2021–22,Manchester United,Round of 16,1–2,Atlético Madrid,1–1 at Wanda Metropolitano0–1 at Old Trafford
141,2021–22,Chelsea,Quarter-finals,4–5 (a.e.t.),Real Madrid,1–3 at Stamford Bridge3–2 at Santiago Bernabéu
142,2021–22,Manchester City,Semi-finals,5–6 (a.e.t.),Real Madrid,4–3 at Etihad Stadium1–3 at Santiago Bernabéu


In [7]:
df_CLseasons = pd.DataFrame(df[3]['Season'],  columns=['Season'])
#print(df_CLseasons)

#Use regex to check for inconsistent text formats and store as 
df_CLseasons['Season'] = df_CLseasons['Season'].str.replace('–','-')
df_CLseasons['DQ_Season'] = df_CLseasons['Season'].str.contains(r'\d\d\d\d\–\d\d$', regex=True)

print(df_CLseasons.loc[df_CLseasons['DQ_Season'] == False])


      Season  DQ_Season
0    1955-56      False
1    1956-57      False
2    1957-58      False
3    1958-59      False
4    1958-59      False
..       ...        ...
139  2020-21      False
140  2021-22      False
141  2021-22      False
142  2021-22      False
143  2021-22      False

[144 rows x 2 columns]


In [8]:
#Before
print(df_CLseasons['Season'][42])
print(df_CLseasons['Season'][53])
print(df_CLseasons['Season'][54])
print(df_CLseasons['Season'][55])

#Correct
df_CLseasons.loc[42,['Season']] = '1990-91'
df_CLseasons.loc[53,['Season']] = '1999-00'
df_CLseasons.loc[54,['Season']] = '1999-00'
df_CLseasons.loc[55,['Season']] = '1999-00'

#After check
print(df_CLseasons['Season'][42])
print(df_CLseasons['Season'][53])
print(df_CLseasons['Season'][54])
print(df_CLseasons['Season'][55])

1990-91[a]
1999-2000
1999-2000
1999-2000
1990-91
1999-00
1999-00
1999-00


In [10]:
zipped = list(zip(df_CLseasons['Season'], df[3]['Club']))


df_all_English_teams =  pd.DataFrame(zipped, columns=['Season','Club'])

In [11]:
print(df_all_English_teams.head(145))

      Season                     Club
0    1955-56             None entered
1    1956-57        Manchester United
2    1957-58        Manchester United
3    1958-59        Manchester United
4    1958-59  Wolverhampton Wanderers
..       ...                      ...
139  2020-21                  Chelsea
140  2021-22        Manchester United
141  2021-22                  Chelsea
142  2021-22          Manchester City
143  2021-22                Liverpool

[144 rows x 2 columns]


In [12]:
#Create a dataframe with CL season unique values, sort columns and then create a PL (qualification) season
# ie this seasons CL entrants qualified from last seasons Premier League final standings.
df_seasons =  pd.DataFrame(df_CLseasons['Season'].unique(), columns=['CL Season'])
df_seasons.sort_values(by=['CL Season'])
df_seasons['PL Season']= df_seasons.shift(1)
df_seasons

,CL Season,PL Season
0,1955-56,NaN
1,1956-57,1955-56
2,1957-58,1956-57
3,1958-59,1957-58
4,1959-60,1958-59
...,...,...
62,2017-18,2016-17
63,2018-19,2017-18
64,2019-20,2018-19
65,2020-21,2019-20


In [13]:
#Create a CL Season to Premier League (PL) qualification mapping dictionary

sm_d={}
# Iterate over column names & create dictionary
for i,j in df_seasons.iterrows():
   sm_d[df_seasons['CL Season'][i]] = df_seasons['PL Season'][i]
print(sm_d)

{'1955-56': nan, '1956-57': '1955-56', '1957-58': '1956-57', '1958-59': '1957-58', '1959-60': '1958-59', '1960-61': '1959-60', '1961-62': '1960-61', '1962-63': '1961-62', '1963-64': '1962-63', '1964-65': '1963-64', '1965-66': '1964-65', '1966-67': '1965-66', '1967-68': '1966-67', '1968-69': '1967-68', '1969-70': '1968-69', '1970-71': '1969-70', '1971-72': '1970-71', '1972-73': '1971-72', '1973-74': '1972-73', '1974-75': '1973-74', '1975-76': '1974-75', '1976-77': '1975-76', '1977-78': '1976-77', '1978-79': '1977-78', '1979-80': '1978-79', '1980-81': '1979-80', '1981-82': '1980-81', '1982-83': '1981-82', '1983-84': '1982-83', '1984-85': '1983-84', '1985-86': '1984-85', '1986-87': '1985-86', '1987-88': '1986-87', '1988-89': '1987-88', '1989-90': '1988-89', '1990-91': '1989-90', '1991-92': '1990-91', '1992-93': '1991-92', '1993-94': '1992-93', '1994-95': '1993-94', '1995-96': '1994-95', '1996-97': '1995-96', '1997-98': '1996-97', '1998-99': '1997-98', '1999-00': '1998-99', '2000-01': '199

In [14]:
df_all_English_teams['PLQ Season']= ''

# Iterate over Champions League Season and add appropriate Premier League Qualification season 
for i,j in  df_all_English_teams.iterrows():
    df_all_English_teams['PLQ Season'][i] = sm_d[df_all_English_teams['Season'][i]]

In [15]:
df_all_English_teams.tail(99)

,Season,Club,PLQ Season
45,1993-94,Manchester United,1992-93
46,1994-95,Manchester United,1993-94
47,1995-96,Blackburn Rovers,1994-95
48,1996-97,Manchester United,1995-96
49,1997-98,Newcastle United,1996-97
...,...,...,...
139,2020-21,Chelsea,2019-20
140,2021-22,Manchester United,2020-21
141,2021-22,Chelsea,2020-21
142,2021-22,Manchester City,2020-21


In [244]:
#Save dataset as csv
df_all_English_teams.to_csv('datasets/ETQ History.csv', index=False)